In [ ]:
!apt-get update
!apt-get install -y g++ openjdk-8-jdk python3-dev   # 필요시
!pip install pandas scikit-learn xgboost transformers sentence-transformers bertopic joblib

# 코랩에서:
from google.colab import drive
drive.mount('/content/drive')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,

In [ ]:
!apt-get update
!pip install "jpype1==1.5.0" konlpy
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

from konlpy.tag import Komoran

komoran = Komoran()
print(komoran.morphs("JPype와 Komoran 테스트 중입니다."))

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
   ━━━━━━━━━━━

In [ ]:
# %% 전역 상수 선언 (config.py)
import re
import pathlib
import torch
from konlpy.tag import Komoran
from sentence_transformers import SentenceTransformer

PROJECT_ROOT = pathlib.Path('/content/drive/MyDrive/IBAS')
DATA_ROOT = PROJECT_ROOT / 'DataSet_DATAUP2025'
RAW_DIR = DATA_ROOT / 'RawData'
CLEANED_DIR = DATA_ROOT / 'Cleaned'
MODELS_DIR = DATA_ROOT / 'Models'
SCORED_DIR = DATA_ROOT / 'Scored'
LABELED_DIR = DATA_ROOT / 'StrongLabeled'
SIGMA = 2
REVIEW_NUM_THRESH = 5
WORD_FREQ_THRESH = 10
RATING_DIFF_THRESH = 1.5
SENTIMENT_DIFF_THRESH = 0.3

VOLUNTARY_PRESUMED_SEED = [
    "서비스", "위생", "청결", "화장실", "응대", "대처", "솔직", "재료"
]
AD_SUBSPICION_SEED = [
    "맛", "맛집", "분위기", "인테리어", "친절", "감성", "플레이팅", "추천"
]
# 광고성 의심 / 자의적 작성 추정 시드 키워드
VOLUNTARY_CONFIRM_SEED = [
    "최악", "실망", "불친절", "다신", "별로", "비추", "아깝다", "불편", "불만", "더럽다", "비위생", "냄새", "절대"
]
AD_CONFIRM_SEED = [
    "협찬", "제공", "원고료", "소정의", "체험단", "대가", "지원", "업체", "포스팅", "무상"
]
# 광고성 확정 / 자의적 작성 확정 시드 키워드
POSITIVE_WORDS_SEED = [
    "맛있","최고","친절","추천","강추","만족","깔끔","재방문","훌륭","감동",
    "좋다","좋았","괜찮","가성비 좋","대박","또 가","또가","기분 좋","정갈","바삭","쫄깃","신선"
]
NEGATIVE_WORDS_SEED = [
    "불친절","최악","별로","실망","다신","비추","비싸기만","늦","지저분","불결","위생",
    "냄새","짜","싱겁","태움","미지근","적","시끄럽","기대 이하","맛없","형편",
    "두번 다시","불만","불쾌","허접","거지같","불편"
]
# 긍정 / 부정 단어 시드 키워드

ALLOWED_PATTERN = re.compile(r'[^가-힣ㄱ-ㅎㅏ-ㅣA-Za-z0-9\\s.,?!]+') # 형태소 분석 모델에서 처리하지 못 하는 문자열
MORPH_ANALZYER = Komoran() # 형태소 분석기
SENTIMENT_POS_TAGS = {"NNG","NNP","VA","VV","VX","MAG","XR"} # 형태소 분석 대상 품사 태그
SBERT_MODEL = SentenceTransformer('jhgan/ko-sbert-sts',
            device = 'cuda' if torch.cuda.is_available() else 'cpu') # SBERT 모델


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# %% 데이터 전처리 (IBAS_DATAUP_DataFilter.py)
import pandas as pd

naver_reviews_raw = pd.read_csv(RAW_DIR / 'Naver_Reviews_Raw.csv', encoding='utf-8-sig')
kakao_reviews_raw = pd.read_csv(RAW_DIR / 'Kakao_Reviews_Raw.csv', encoding='utf-8-sig')
print(naver_reviews_raw.shape)
print(kakao_reviews_raw.shape)
# 네이버/카카오맵 리뷰 원본 데이터 불러오기

naver_reviews = naver_reviews_raw[['도입_URL', '가게이름', '카테고리', '전체평점', '방문자리뷰', '리뷰작성자',
                                   '리뷰작성수', '리뷰내용', '방문시간']].copy()
kakao_reviews = kakao_reviews_raw[['가게이름', '카테고리', '전체평점', '평점건수', '리뷰수', '리뷰작성자', '리뷰작성일',
                                   '작성자리뷰작성수', '작성자평균평점', '리뷰평점', '리뷰내용']].copy()
# 피쳐 선정

kakao_reviews['평점건수'] = kakao_reviews['평점건수'].str.replace(r'[\r\n]', '', regex=True)
kakao_reviews['리뷰수'] = kakao_reviews['리뷰수'].str.replace(r'[\r\n]', '', regex=True)
kakao_reviews['리뷰작성자'] = kakao_reviews['리뷰작성자'].str.replace(r'[\r\n]', '', regex=True)
kakao_reviews['리뷰내용'] = kakao_reviews['리뷰내용'].str.replace(r'[\r\n]', '', regex=True)
# 카카오 리뷰 데이터에서 블필요한 이스케이프 문자 제거
naver_reviews['리뷰작성수'] = pd.to_numeric(naver_reviews['리뷰작성수'], errors='coerce')
naver_reviews['방문자리뷰'] = pd.to_numeric(naver_reviews['방문자리뷰'], errors='coerce')
# 네이버 리뷰 데이터 리뷰작성수 및 방문자 리뷰 데이터 정수형으로 변환

def missing_remover (df_naver_raw, df_kakao_raw = None) :
    df_naver_raw.dropna(subset=['가게이름', '방문자리뷰', '리뷰작성자', '리뷰내용'], inplace=True)
    df_naver_raw['카테고리'] = df_naver_raw['카테고리'].fillna('')
    df_naver_raw['전체평점'] = df_naver_raw['전체평점'].fillna('미공개')
    df_naver_raw['리뷰작성수'] = df_naver_raw['리뷰작성수'].fillna(1)
    # 네이버 리뷰 데이터 결측치 제거 및 대체
    df_kakao_raw.dropna(subset=['리뷰작성자'], inplace=True)
    df_kakao_raw.dropna(subset=['평점건수', '리뷰수'], how='all', inplace=True)
    df_kakao_raw['전체평점'] = df_kakao_raw['전체평점'].fillna('평점 없음')
    df_kakao_raw['평점건수'] = df_kakao_raw['평점건수'].fillna(0)
    df_kakao_raw['리뷰수'] = df_kakao_raw['리뷰수'].fillna(0)
    df_kakao_raw['리뷰내용'] = df_kakao_raw['리뷰내용'].fillna('내용없음')
    # 카카오 리뷰 데이터 결측치 제거 및 대체
    return df_naver_raw, df_kakao_raw
naver_reviews, kakao_reviews = missing_remover(naver_reviews, kakao_reviews)
# 결측치 처리

kakao_reviews['평점건수'] = kakao_reviews['평점건수'].str.replace('개', '', regex=False)
kakao_reviews['리뷰수'] = kakao_reviews['리뷰수'].str.replace('개', '', regex=False)
kakao_reviews['평점건수'] = kakao_reviews['평점건수'].fillna(0)
kakao_reviews['리뷰수'] = kakao_reviews['리뷰수'].fillna(0)
kakao_reviews['평점건수'] = kakao_reviews['평점건수'].astype(int)
kakao_reviews['리뷰수'] = kakao_reviews['리뷰수'].astype(int)
# 카카오 리뷰 데이터 평점건수 및 리뷰수 데이터 정수형으로 변환

print(naver_reviews.shape)
print(kakao_reviews.shape)
naver_reviews.to_csv(CLEANED_DIR / 'Naver_Reviews.csv', index=False, encoding='utf-8-sig')
kakao_reviews.to_csv(CLEANED_DIR / 'Kakao_Reviews.csv', index=False, encoding='utf-8-sig')
# 전처리 완료 데이터 CSV로 저장

/tmp/ipython-input-2427903705.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  naver_reviews_raw = pd.read_csv(RAW_DIR / 'Naver_Reviews_Raw.csv', encoding='utf-8-sig')


(41332, 16)
(1903, 21)
(30192, 9)
(1841, 11)


In [ ]:
# %% 플랫폼 신뢰도 기반 가게 필터링 시스템 (IBAS_DATAUP_NaverFilter.py)
# 1. SBERT 기반 감성 사전 구축 및 감성 지표 계산
import pandas as pd
import numpy as np
import re
import torch
from sentence_transformers import SentenceTransformer
from collections import Counter
from typing import List, Dict, Tuple
import pickle

def _norm_store_name(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s)
    s = re.sub(ALLOWED_PATTERN, " ", s)   # 괄호, 비한글/영문/숫자 제거
    return s.lower().strip()
# 가게 이름에서 비정상적인 문자열 제거
def extract_candidate_terms(
    dfs: list[pd.DataFrame],
    text_col: str = "리뷰내용",
    max_token_len: int = 10,
) -> list[str]:
    counter = Counter()

    for df in dfs:
        s = df[text_col].fillna("").astype(str)
        for txt in s:
            tokens = komoran_tokenize(txt)  # Komoran 기반 토큰화
            for tok in tokens:
                if len(tok) <= max_token_len:
                    counter[tok] += 1

    candidates = [t for t, c in counter.items() if c >= WORD_FREQ_THRESH]
    return candidates
# 리뷰 데이터에서 후보 단어군 추출
def komoran_tokenize(text: str) -> list[str]:
    if not isinstance(text, str):
        return []

    # 1) 문자열 강제 변환 + 깨지는 문자 제거
    text = str(text)
    # ALLOWED_PATTERN: 이미 코드에 정의되어 있음
    text = ALLOWED_PATTERN.sub(" ", text)

    # 2) 유니코드 정제 (깨지는 바이트 무시)
    try:
        # 만약 text가 이미 str인데도 내부에 깨진 부분이 있다면
        # encode/decode 과정에서 정리
        text = text.encode("utf-8", "ignore").decode("utf-8", "ignore")
    except Exception:
        raise Exception("Unicode Error!")

    text = text.strip()
    if not text:
        return []

    morphs = []
    for morph, pos in MORPH_ANALZYER.pos(text):
        if pos in SENTIMENT_POS_TAGS:
            morphs.append(morph)
    return morphs
# Komoran 형태소 분석 결과에서 감성에 관련된 품사만 추출해 토큰 리스트 반환
def build_sbert_sentiment_lexicon(
    naver_df: pd.DataFrame,
    kakao_df: pd.DataFrame,
    model: SentenceTransformer,
    sim_threshold: float = 0.30,
    max_candidates: int = 50000,
) -> tuple[dict[str, float], dict[str, float]]:
    # 1) 후보 토큰 추출 (Komoran + WORD_FREQ_THRESH 기반)
    candidates = extract_candidate_terms(
        dfs=[naver_df, kakao_df],
        text_col="리뷰내용",
        max_token_len=10,
    )
    # 후보 수 제한
    if max_candidates is not None and len(candidates) > max_candidates:
        candidates = candidates[:max_candidates]

    # 2) Seed / 후보 임베딩
    pos_seeds = POSITIVE_WORDS_SEED
    neg_seeds = NEGATIVE_WORDS_SEED

    pos_emb = model.encode(pos_seeds, convert_to_tensor=True, batch_size=64, show_progress_bar=True)
    neg_emb = model.encode(neg_seeds, convert_to_tensor=True, batch_size=64, show_progress_bar=True)
    cand_emb = model.encode(candidates, convert_to_tensor=True, batch_size=64, show_progress_bar=True)

    pos_emb = torch.nn.functional.normalize(pos_emb, p=2, dim=1)
    neg_emb = torch.nn.functional.normalize(neg_emb, p=2, dim=1)
    cand_emb = torch.nn.functional.normalize(cand_emb, p=2, dim=1)

    sim_pos = torch.mm(cand_emb, pos_emb.T)  # [C, P]
    sim_neg = torch.mm(cand_emb, neg_emb.T)  # [C, N]

    sim_pos_max, _ = sim_pos.max(dim=1)
    sim_neg_max, _ = sim_neg.max(dim=1)

    scores = (sim_pos_max - sim_neg_max).cpu().numpy()  # >0: 긍정, <0: 부정

    pos_lexicon: dict[str, float] = {}
    neg_lexicon: dict[str, float] = {}

    for term, sc in zip(candidates, scores):
        if abs(sc) < sim_threshold:
            continue
        if sc > 0:
            pos_lexicon[term] = float(sc)   # 긍정 weight
        else:
            neg_lexicon[term] = float(-sc)  # 부정 weight(양수)

    return pos_lexicon, neg_lexicon
def save_sentiment_lexicon(
    pos_lexicon: dict[str,float],
    neg_lexicon: dict[str,float],
    pos_path: pathlib.Path = MODELS_DIR / "pos_lexicon.pkl",
    neg_path: pathlib.Path = MODELS_DIR / "neg_lexicon.pkl",
) -> None:
    with open(pos_path, "wb") as f:
        pickle.dump(pos_lexicon, f)
    with open(neg_path, "wb") as f:
        pickle.dump(neg_lexicon, f)
def load_sentiment_lexicon(
    pos_path: pathlib.Path = MODELS_DIR / "pos_lexicon.pkl",
    neg_path: pathlib.Path = MODELS_DIR / "neg_lexicon.pkl",
) -> tuple[dict[str,float], dict[str,float]]:
    with open(pos_path, "rb") as f:
        pos_lexicon = pickle.load(f)
    with open(neg_path, "rb") as f:
        neg_lexicon = pickle.load(f)
    return pos_lexicon, neg_lexicon
# SBERT 기반 감성 사전 구축/저장/로딩 유틸
def sentiment_per_review(
    text_series: pd.Series,
    pos_lexicon: dict[str, float],
    neg_lexicon: dict[str, float],
    min_score_eps: float = 1e-6,
) -> tuple[pd.Series, pd.Series, pd.Series, pd.Series, pd.Series]:
    """
    반환:
      sent_score: pos_score - neg_score
      pos_ratio:  pos_score / (pos_score + neg_score + eps)
      neg_ratio:  neg_score / (pos_score + neg_score + eps)
      pos_flag:   긍정 토큰 존재 여부 (0/1)
      neg_flag:   부정 토큰 존재 여부 (0/1)
    """
    s = text_series.fillna("").astype(str)
    sent_scores, pos_ratios, neg_ratios, pos_flags, neg_flags = [], [], [], [], []
    for txt in s:
        tokens = komoran_tokenize(txt)
        pos_score = 0.0
        neg_score = 0.0
        for tok in tokens:
            if tok in pos_lexicon:
                pos_score += pos_lexicon[tok]
            if tok in neg_lexicon:
                neg_score += neg_lexicon[tok]

        total = pos_score + neg_score

        if total < min_score_eps:
            pos_ratio = 0.0
            neg_ratio = 0.0
            pos_flag = 0
            neg_flag = 0
        else:
            pos_ratio = pos_score / total
            neg_ratio = neg_score / total
            pos_flag = int(pos_score > 0.0)
            neg_flag = int(neg_score > 0.0)

        sent_scores.append(pos_score - neg_score)
        pos_ratios.append(pos_ratio)
        neg_ratios.append(neg_ratio)
        pos_flags.append(pos_flag)
        neg_flags.append(neg_flag)

    sent_scores = pd.Series(sent_scores, index=s.index, dtype=float)
    pos_ratios  = pd.Series(pos_ratios,  index=s.index, dtype=float)
    neg_ratios  = pd.Series(neg_ratios,  index=s.index, dtype=float)
    pos_flags   = pd.Series(pos_flags,   index=s.index, dtype=int)
    neg_flags   = pd.Series(neg_flags,   index=s.index, dtype=int)

    return sent_scores, pos_ratios, neg_ratios, pos_flags, neg_flags
# Komoran으로 토큰화한 뒤 SBERT‑확장 lexicon으로 리뷰별 감성 지표 계산.

# 2. 긍정/부정 감성 지표 기반 가게 필터링
def filter_naver_reviews(
    df_naver,
    df_kakao,
    pos_lexicon: Dict[str, float],
    neg_lexicon: Dict[str, float],
    # 감성 대비 기준 (SBERT-lexicon 기반 store-level 지표)
    min_reviews_sent: int = 5,
    kakao_neg_mean_max: float = -0.10,
    kakao_neg_share_min: float = 0.50,
    naver_pos_mean_min: float = 0.30,
    naver_pos_share_min: float = 0.65,
    store_col: str = "가게이름",
    text_col: str = "리뷰내용",
    rating_col: str = "전체평점",
) -> Tuple[pd.DataFrame, dict]:

    # ---- 0) 복사 및 가게 키 정규화 ----
    ndf = df_naver.copy()
    kdf = df_kakao.copy()
    ndf["_store_key"] = ndf[store_col].map(_norm_store_name)
    kdf["_store_key"] = kdf[store_col].map(_norm_store_name)

    # ---- 1) 평점 기반 가게 삭제 ----
    def _safe_num(x):
        return pd.to_numeric(x, errors="coerce")

    n_grp = ndf.groupby("_store_key").agg(
        n_avg=(rating_col, lambda x: _safe_num(x).mean()),
        n_cnt=(rating_col, "size"),
    )
    if "평점건수" in kdf.columns:
        k_grp = kdf.groupby("_store_key").agg(
            k_avg=(rating_col, lambda x: _safe_num(x).mean()),
            k_cnt=("평점건수", lambda x: _safe_num(x).mean()),
        )
        k_cnt_fb = kdf.groupby("_store_key")[rating_col].size().rename("k_cnt_fb")
        k_grp = k_grp.join(k_cnt_fb, how="left")
        k_grp["k_cnt"] = k_grp["k_cnt"].fillna(k_grp["k_cnt_fb"])
        k_grp = k_grp.drop(columns=["k_cnt_fb"])
    else:
        k_grp = kdf.groupby("_store_key").agg(
            k_avg=(rating_col, lambda x: _safe_num(x).mean()),
            k_cnt=(rating_col, "size"),
        )
    merged_rating = n_grp.join(k_grp, how="inner")
    cond_cnt = (merged_rating["n_cnt"] >= REVIEW_NUM_THRESH) & \
               (merged_rating["k_cnt"] >= REVIEW_NUM_THRESH)
    cond_rating = (merged_rating["n_avg"] - merged_rating["k_avg"]).abs() >= RATING_DIFF_THRESH
    bad_stores_rating = set(merged_rating.index[cond_cnt & cond_rating])

    # ---- 2) 리뷰 단위 Komoran+SBERT-lexicon 감성 지표 계산 ----
    # 네이버
    n_sent_score, n_pos_ratio, n_neg_ratio, n_pos_flag, n_neg_flag = \
        sentiment_per_review(
            ndf[text_col], pos_lexicon, neg_lexicon
        )
    ndf["_sent_score"] = n_sent_score
    ndf["_pos_flag"]   = n_pos_flag
    ndf["_neg_flag"]   = n_neg_flag

    # 카카오
    k_sent_score, k_pos_ratio, k_neg_ratio, k_pos_flag, k_neg_flag = \
        sentiment_per_review(
            kdf[text_col], pos_lexicon, neg_lexicon
        )
    kdf["_sent_score"] = k_sent_score
    kdf["_pos_flag"]   = k_pos_flag
    kdf["_neg_flag"]   = k_neg_flag

    # ---- 3) 가게 단위 감성 집계 ----
    n_store_sent = ndf.groupby("_store_key").agg(
        sent_mean_n=("_sent_score","mean"),
        pos_share_n=("_pos_flag","mean"),
        neg_share_n=("_neg_flag","mean"),
        sent_n_n   =("_sent_score","size"),
    )
    k_store_sent = kdf.groupby("_store_key").agg(
        sent_mean_k=("_sent_score","mean"),
        pos_share_k=("_pos_flag","mean"),
        neg_share_k=("_neg_flag","mean"),
        sent_n_k   =("_sent_score","size"),
    )
    sj = k_store_sent.join(n_store_sent, how="inner")
    # 최소 리뷰 수 기준
    sj = sj[(sj["sent_n_k"] >= min_reviews_sent) & (sj["sent_n_n"] >= min_reviews_sent)]
    # 카카오 부정, 네이버 과긍정 기준
    kakao_negative = (sj["sent_mean_k"] <= kakao_neg_mean_max) & \
                     (sj["neg_share_k"] >= kakao_neg_share_min)
    naver_overpos  = (sj["sent_mean_n"] >= naver_pos_mean_min) & \
                     (sj["pos_share_n"] >= naver_pos_share_min)
    bad_stores_sent = set(sj.index[kakao_negative & naver_overpos])

    # ---- 4) 최종 가게 삭제 집합 및 네이버 필터링 ----
    bad_union = bad_stores_rating.union(bad_stores_sent)
    naver_filtered = ndf[~ndf["_store_key"].isin(bad_union)].drop(
        columns=["_store_key","_sent_score","_pos_flag","_neg_flag"],
        errors="ignore"
    )
    stats = {
        "stores_removed_by_rating": len(bad_stores_rating),
        "stores_removed_by_sent": len(bad_stores_sent),
        "stores_removed_union": len(bad_union),
        "naver_before": len(ndf),
        "naver_after": len(naver_filtered),
    }
    return naver_filtered, stats
"""
네이버/카카오 리뷰를 입력으로 받아 평점 차이 기준 및 Komoran + SBERT-lexicon 기반 감성 지표 기준
으로 가게 단위 필터링을 수행하고, 필터링된 네이버 리뷰와 통계 정보를 반환
"""

# 전처리 완료 데이터 로드
naver_reviews = pd.read_csv(CLEANED_DIR / 'Naver_Reviews.csv', encoding='utf-8-sig')
kakao_reviews = pd.read_csv(CLEANED_DIR / 'Kakao_Reviews.csv', encoding='utf-8-sig')

# SBERT 감성 사전 로딩 또는 구축
pos_lex_path = MODELS_DIR / "pos_lexicon.pkl"
neg_lex_path = MODELS_DIR / "neg_lexicon.pkl"

if pos_lex_path.exists() and neg_lex_path.exists():
    print("[Sentiment Lexicon] 기존 감성 사전 로딩 중...")
    pos_lexicon, neg_lexicon = load_sentiment_lexicon(pos_lex_path, neg_lex_path)
    print(f"  - pos_lexicon: {len(pos_lexicon)} terms")
    print(f"  - neg_lexicon: {len(neg_lexicon)} terms")
else:
    print("[Sentiment Lexicon] 감성 사전이 없어 새로 구축합니다...")
    pos_lexicon, neg_lexicon = build_sbert_sentiment_lexicon(
        naver_df=naver_reviews,
        kakao_df=kakao_reviews,
        model=SBERT_MODEL,
        sim_threshold=0.30,
        max_candidates=50000,
    )
    save_sentiment_lexicon(pos_lexicon, neg_lexicon, pos_lex_path, neg_lex_path)
    print(f"  - pos_lexicon: {len(pos_lexicon)} terms 저장 완료")
    print(f"  - neg_lexicon: {len(neg_lexicon)} terms 저장 완료")

# 플랫폼 신뢰도 기반 네이버 필터링 실행
naver_reviews_filtered, filter_stats = filter_naver_reviews(
    df_naver=naver_reviews,
    df_kakao=kakao_reviews,
    pos_lexicon=pos_lexicon,
    neg_lexicon=neg_lexicon,
)
print(filter_stats)
naver_reviews_filtered.to_csv(CLEANED_DIR / 'Naver_Reviews_Filtered.csv', encoding='utf-8-sig', index=False)

[Sentiment Lexicon] 감성 사전이 없어 새로 구축합니다...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/33 [00:00<?, ?it/s]

  - pos_lexicon: 14 terms 저장 완료
  - neg_lexicon: 11 terms 저장 완료
{'stores_removed_by_rating': 5, 'stores_removed_by_sent': 0, 'stores_removed_union': 5, 'naver_before': 30192, 'naver_after': 29229}


In [ ]:
# %% 리뷰 내용 감성 분석 실시 (IBAS_DATAUP_SentimentCalculator.py)
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[generic] 감성 분석 모델 로드 중 (Device: {device})...")

MODEL_NAME = "alsgyu/sentiment-analysis-fine-tuned-model"

generic_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
generic_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
generic_model.to(device)
generic_model.eval()

print(f"[generic] 감성 분석 모델 로드 완료")
print(f" - 모델: {MODEL_NAME}")
print(f" - 클래스 수: {generic_model.config.num_labels}")
print(f" - 레이블 매핑: {generic_model.config.id2label}")

# 2. X3 계산 함수
def _score_generic(texts: list[str]) -> list[float]:
    inputs = generic_tokenizer(
        texts,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding=True,
    ).to(device)

    with torch.no_grad():
        outputs = generic_model(**inputs)  # generic_model은 이미 .to(device) 되어 있음

    logits = outputs.logits                         # [B, num_labels]
    probs = torch.nn.functional.softmax(logits, dim=-1)  # [B, 3] 가정
    neg = probs[:, 0]
    pos = probs[:, 1]
    neu = probs[:, 2]
    scores = (pos * 1.0 + neu * 0.5 + neg * 0.0).detach().cpu().tolist()
    return scores
# 범용 3-class 모델로부터 X3 점수 계산, X3 = pos_prob*1.0 + neu_prob*0.5 + neg_prob*0.0
def calculate_sentiment_score(text: str, mode: str = "generic", debug: bool = False) -> float:
    """
    단일 텍스트에 대해 X3를 계산.
      - mode='generic' : 범용 3-class 모델 기반 (pos=1, neu=0.5, neg=0)
      - mode='domain'  : KR3 기반 2-class 모델, X3=긍정 확률
    """
    if pd.isna(text) or len(str(text)) <= 10:
        return 0.5  # 짧은 텍스트는 중립으로 처리

    try:
        if mode == "generic":
            scores = _score_generic([str(text)])
        elif mode == "domain":
            scores = _score_domain([str(text)])
        else:
            raise ValueError("mode must be 'generic' or 'domain'")
        score = float(scores[0])

        if debug:
            print(f"\n[DEBUG-{mode}] Text: {text[:50]}...")
            print(f"[DEBUG-{mode}] X3 score: {score:.3f}\n")

        return score

    except Exception as e:
        print(f"Warning: Sentiment analysis failed ({mode}): {e}")
        print(f"  Text: {str(text)[:50]}...")
        import traceback
        traceback.print_exc()
        return 0.5
def apply_sentiment_batch(df, platform_name: str, mode: str = "generic") -> pd.DataFrame:
    """
    DataFrame 단위 X3 계산:
      - mode='generic' : 기존 OpenVINO 범용 모델
      - mode='domain'  : KR3 기반 도메인 모델
    """
    print(f"\n[{platform_name}] 감성 분석(X3, mode={mode}) 배치 계산 시작...")

    # 첫 번째 샘플로 디버그 출력
    if len(df) > 0:
        print(f"\n{'=' * 60}")
        print(f"[레이블 확인] 첫 번째 리뷰로 테스트 중...")
        first_text = df['리뷰내용'].iloc[0]
        test_score = calculate_sentiment_score(first_text, mode=mode, debug=True)
        print(f"[레이블 확인] 테스트 완료. 점수: {test_score:.3f}")
        print(f"{'=' * 60}\n")

    try:
        review_texts_list = df['리뷰내용'].fillna('').astype(str).tolist()
        sentiment_scores = []
        batch_size = 32

        pbar = tqdm(total=len(review_texts_list),
                    desc=f"[{platform_name}] 감성 분석 ({mode})",
                    unit="리뷰")

        for i in range(0, len(review_texts_list), batch_size):
            batch_texts = review_texts_list[i:i + batch_size]

            valid_texts = []
            valid_indices = []
            batch_scores = [0.5] * len(batch_texts)

            for idx, text in enumerate(batch_texts):
                if len(text) > 10:
                    valid_texts.append(text)
                    valid_indices.append(idx)

            if valid_texts:
                try:
                    if mode == "generic":
                        scores = _score_generic(valid_texts)
                    elif mode == "domain":
                        scores = _score_domain(valid_texts)
                    else:
                        raise ValueError("mode must be 'generic' or 'domain'")

                    for j, valid_idx in enumerate(valid_indices):
                        batch_scores[valid_idx] = float(scores[j])

                except Exception as e:
                    print(f"\n  배치 처리 실패 (mode={mode}, 위치: {i}): {e}. 개별 처리로 전환...")
                    for idx, text in enumerate(batch_texts):
                        if len(text) > 10:
                            batch_scores[idx] = calculate_sentiment_score(text, mode=mode)

            sentiment_scores.extend(batch_scores)
            pbar.update(len(batch_texts))

        pbar.close()
        df['X3_Sentiment'] = sentiment_scores

    except Exception as e:
        print(f"Warning: Batch Sentiment analysis failed (mode={mode}): {e}. 개별 처리로 전환...")
        df['X3_Sentiment'] = df['리뷰내용'].apply(
            lambda x: calculate_sentiment_score(x, mode=mode)
        )

    df['X3_Sentiment'] = df['X3_Sentiment'].fillna(0.5)

    print(f"\n[{platform_name}] 감성 분석(X3, mode={mode}) 계산 완료.")
    print(f"  - 평균 감성 점수: {df['X3_Sentiment'].mean():.3f}")
    print(f"  - 표준편차: {df['X3_Sentiment'].std():.3f}")
    print(f"  - 최소값: {df['X3_Sentiment'].min():.3f}")
    print(f"  - 최대값: {df['X3_Sentiment'].max():.3f}")

    return df

# 3. 데이터 로드 및 감성 분석 적용
print("\n데이터 로드 중...")
naver_reviews_filtered = pd.read_csv(CLEANED_DIR / 'Naver_Reviews_Filtered.csv', encoding='utf-8-sig')
kakao_reviews          = pd.read_csv(CLEANED_DIR / 'Kakao_Reviews.csv',           encoding='utf-8-sig')

naver_reviews_generic = apply_sentiment_batch(naver_reviews_filtered.copy(), 'Naver', mode="generic")
kakao_reviews_generic = apply_sentiment_batch(kakao_reviews.copy(),          'Kakao', mode="generic")
print("\n결과 저장 중...")
naver_reviews_generic.to_csv(CLEANED_DIR / 'Naver_Reviews_Filtered.csv', index=False, encoding='utf-8-sig')
kakao_reviews_generic.to_csv(CLEANED_DIR / 'Kakao_Reviews.csv', index=False, encoding='utf-8-sig')
print("✓ 모든 리뷰 감성 분석(X3, 도메인 모델) 완료 및 저장됨.")

[generic] 감성 분석 모델 로드 중 (Device: cuda)...
[generic] 감성 분석 모델 로드 완료
 - 모델: alsgyu/sentiment-analysis-fine-tuned-model
 - 클래스 수: 3
 - 레이블 매핑: {0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

데이터 로드 중...

[Naver] 감성 분석(X3, mode=generic) 배치 계산 시작...

[레이블 확인] 첫 번째 리뷰로 테스트 중...

[DEBUG-generic] Text: 항강 지나다니던 인하대 골목에서 여러번 지나치다가
드디어 오게됫어요!
고기들이 양도 넉넉하...
[DEBUG-generic] X3 score: 0.510

[레이블 확인] 테스트 완료. 점수: 0.510



[Naver] 감성 분석 (generic): 100%|██████████| 29229/29229 [02:00<00:00, 241.68리뷰/s]



[Naver] 감성 분석(X3, mode=generic) 계산 완료.
  - 평균 감성 점수: 0.515
  - 표준편차: 0.063
  - 최소값: 0.080
  - 최대값: 0.943

[Kakao] 감성 분석(X3, mode=generic) 배치 계산 시작...

[레이블 확인] 첫 번째 리뷰로 테스트 중...

[DEBUG-generic] Text: 가성비가 너무 좋아서 종종 시켜먹는데 맛있어요!기본으로 나오는 스프도 고소하고 담백한게 맛...
[DEBUG-generic] X3 score: 0.504

[레이블 확인] 테스트 완료. 점수: 0.504



[Kakao] 감성 분석 (generic): 100%|██████████| 1841/1841 [00:04<00:00, 396.06리뷰/s]



[Kakao] 감성 분석(X3, mode=generic) 계산 완료.
  - 평균 감성 점수: 0.525
  - 표준편차: 0.128
  - 최소값: 0.082
  - 최대값: 0.937

결과 저장 중...
✓ 모든 리뷰 감성 분석(X3, 도메인 모델) 완료 및 저장됨.


In [ ]:
# %% 광고성 리뷰 필터링 - 피쳐 엔지니어링 (IBAS_DATAUP_FeatureEngineer.py)
# 오프라인: SBERT 후보 추출 및 키워드 확장
import pandas as pd
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pickle

def build_candidate_embeddings(
    naver_df: pd.DataFrame,
    kakao_df: pd.DataFrame,
    analyzer,
    pattern,
    model,
    min_frequency: int,
    batch_size: int = 64,
):
    print("Komoran으로 전체 명사 리스트 추출 시작...")
    all_review_texts = pd.concat([
        naver_df['리뷰내용'],
        kakao_df['리뷰내용'],
    ]).dropna()

    all_nouns_list = []
    for text in all_review_texts:
        try:
            cleaned_text = pattern.sub(r' ', str(text))
            stripped_text = cleaned_text.strip()
            if not stripped_text:
                continue
            nouns = analyzer.nouns(stripped_text)
            all_nouns_list.extend(nouns)
        except Exception as e:
            print(f"Warning: Noun extraction failed: {e}")
            continue

    noun_counts = Counter(all_nouns_list)
    candidate_words = [
        word for word, count in noun_counts.items()
        if count >= min_frequency and len(word) > 1
    ]
    print(f"전체 고유 명사: {len(noun_counts)}개 -> {min_frequency}회 이상: {len(candidate_words)}개")

    del all_review_texts, all_nouns_list, noun_counts

    print(f"후보 단어 {len(candidate_words)}개 임베딩 생성 중... (배치 크기: {batch_size})")
    candidate_embeddings = model.encode(
        candidate_words,
        batch_size=batch_size,
        show_progress_bar=True,
    )
    print("임베딩 생성 완료.")
    return candidate_words, candidate_embeddings
def get_similar_keywords_from_list(
    seed_keywords_list,
    words,
    embeddings,
    model,
    top_n: int = 15,
    sim_threshold: float = 0.30,
):
    seed_embeddings = model.encode(seed_keywords_list)
    sim_matrix = cosine_similarity(seed_embeddings, embeddings)  # [S, C]
    max_similarities = np.max(sim_matrix, axis=0)               # [C]

    results = []
    seed_set = set(seed_keywords_list)
    for i, word in enumerate(words):
        if word in seed_set:
            continue
        score = float(max_similarities[i])
        if score < sim_threshold:
            continue
        results.append((word, score))

    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_n]
# seed 리스트와 후보 임베딩을 받아, 각 후보에 대해 최대 코사인 유사도 계산, sim >= sim_threshold인 것만 남기고, 상위 top_n을 반환
naver_reviews_filtered = pd.read_csv(CLEANED_DIR / 'Naver_Reviews_Filtered.csv', encoding='utf-8-sig')
kakao_reviews = pd.read_csv(CLEANED_DIR / 'Kakao_Reviews.csv', encoding='utf-8-sig')
candidate_words, candidate_embeddings = build_candidate_embeddings(
    naver_reviews_filtered,
    kakao_reviews,
    MORPH_ANALZYER,
    ALLOWED_PATTERN,
    SBERT_MODEL,
    min_frequency=WORD_FREQ_THRESH,
)
VOLUNTARY_PRESUMED_EXPAND_TUPLES = get_similar_keywords_from_list(
    VOLUNTARY_PRESUMED_SEED,
    candidate_words,
    candidate_embeddings,
    SBERT_MODEL,
    top_n=30,
    sim_threshold=0.30,
)
AD_SUBSPICION_EXPAND_TUPLES = get_similar_keywords_from_list(
    AD_SUBSPICION_SEED,
    candidate_words,
    candidate_embeddings,
    SBERT_MODEL,
    top_n=30,
    sim_threshold=0.30,
)
VOLUNTARY_PRESUMED_EXPAND = [w for w, _ in VOLUNTARY_PRESUMED_EXPAND_TUPLES]
AD_SUBSPICION_EXPAND = [w for w, _ in AD_SUBSPICION_EXPAND_TUPLES]
with open(MODELS_DIR / "voluntary_keywords.pkl", "wb") as f:
    pickle.dump(VOLUNTARY_PRESUMED_EXPAND, f)
with open(MODELS_DIR / "ad_keywords.pkl", "wb") as f:
    pickle.dump(AD_SUBSPICION_EXPAND, f)
print("✓ 키워드 확장 결과 저장 완료.")

Komoran으로 전체 명사 리스트 추출 시작...
전체 고유 명사: 6336개 -> 10회 이상: 1189개
후보 단어 1189개 임베딩 생성 중... (배치 크기: 64)


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

임베딩 생성 완료.
✓ 키워드 확장 결과 저장 완료.


In [ ]:
# %% 온라인: 광고성 리뷰 피쳐 계산
import pandas as pd
import re
from datetime import datetime
from collections import Counter
import pickle
import warnings
warnings.filterwarnings('ignore')

def parse_naver_visit_time_to_dt(s: str) -> datetime:
    """
    네이버 '방문시간' 컬럼을 'YYYY.MM.DD' 형태의 datetime으로 변환.
    - 올해: 'MM.DD.요일'
    - 과거: 'YYYY.MM.DD.요일'
    """
    s = str(s).strip()
    if not s:
        return pd.NaT

    parts = s.split(".")
    try:
        if len(parts) == 3:
            # 'MM.DD.요일' → 올해 연도 가정
            month = int(parts[0])
            day   = int(parts[1])
            year  = datetime.now().year
        elif len(parts) >= 4:
            # 'YYYY.MM.DD.요일'
            year  = int(parts[0])
            month = int(parts[1])
            day   = int(parts[2])
        else:
            return pd.NaT
        # 여기서 datetime으로 만든 뒤, 포맷은 'YYYY.MM.DD'로 맞추고 싶으면
        # Grouper 등에서 datetime 타입이 필요하므로, 최종적으로는 datetime을 반환
        return datetime.strptime(f"{year:04d}.{month:02d}.{day:02d}", "%Y.%m.%d")
    except Exception:
        return pd.NaT
def parse_kakao_review_date_to_dt(s: str) -> datetime:
    """
    카카오 '리뷰작성일' 컬럼을 'YYYY.MM.DD' 형태의 datetime으로 변환.
    - 형식: 'YYYY.MM.DD'
    """
    s = str(s).strip()
    if not s:
        return pd.NaT
    try:
        return datetime.strptime(s, "%Y.%m.%d")
    except Exception:
        digits = re.findall(r"\d+", s)
        if len(digits) >= 3:
            year  = int(digits[0])
            month = int(digits[1])
            day   = int(digits[2])
            return datetime.strptime(f"{year:04d}.{month:02d}.{day:02d}", "%Y.%m.%d")
        return pd.NaT

def calculate_jaccard_similarity(texts):
    try:
        if isinstance(texts, pd.Series):
            texts = texts.tolist()
        texts = [str(t).strip() for t in texts if t and str(t).strip()]

        if len(texts) < 2:
            return 0.0

        text_sets = [set(str(t).lower().split()) for t in texts]

        sims = []
        n = len(text_sets)
        for i in range(n):
            for j in range(i + 1, n):
                s1, s2 = text_sets[i], text_sets[j]
                if not s1 and not s2:
                    sims.append(1.0)
                elif not s1 or not s2:
                    sims.append(0.0)
                else:
                    inter = len(s1 & s2)
                    union = len(s1 | s2)
                    sims.append(inter / union if union > 0 else 0.0)

        return float(np.mean(sims)) if sims else 0.0
    except Exception as e:
        print(f"Warning: Jaccard similarity calculation failed: {e}")
        return -1.0
def calculate_content_similarity(texts):
    try:
        if isinstance(texts, pd.Series):
            texts = texts.tolist()
        texts = [str(t).strip() for t in texts if t and str(t).strip()]
        if len(texts) < 2:
            return 0.0
        return calculate_jaccard_similarity(texts)
    except Exception as e:
        print(f"Warning: Could not calculate content similarity: {e}")
        return -1.0
# X7: 유저 리뷰 일관성, simple Jaccard n-gram similarity만 사용, 리뷰 텍스트 집합 전체의 평균 Jaccard 유사도 반환.
def count_keywords(text, keyword_set, analyzer):
    if pd.isna(text) or not text:
        return 0
    try:
        cleaned_text = ALLOWED_PATTERN.sub(r' ', str(text))
        stripped_text = cleaned_text.strip()
        if not stripped_text:
            return 0

        nouns = analyzer.nouns(stripped_text)
        cnt = 0
        for noun in nouns:
            if noun in keyword_set:
                cnt += 1
        return cnt
    except Exception as e:
        print(f"Warning: Keyword count analysis failed: {e}")
        return 0
# 텍스트에서 특정 명사 키워드의 개수 계산
def feature_calculator(df_filtered, sigma, platform_name, ad_keywords, voluntary_keywords):
    df_temp = df_filtered.copy()

    if platform_name == 'naver':
        date_col_name = '방문시간'
        user_groups = df_temp.groupby('리뷰작성자')
        user_summary = user_groups.agg(
            X5_rating_mean=('리뷰작성자', lambda x: 3.0),
            X6_review_num=('리뷰작성수', 'first'),
        )
        print(f"[{platform_name}] 유저(X5, X6) 정보 집계 완료.")
        user_texts = user_groups['리뷰내용'].apply(list)
        print(f"[{platform_name}] 유저(X7) 유사도(Jaccard) 계산 시작...")
        user_summary['X7_content_sim'] = user_texts.apply(calculate_content_similarity)
        print(f"[{platform_name}] 유저(X7) 유사도(Jaccard) 계산 완료.")
    elif platform_name == 'kakao':
        date_col_name = '리뷰작성일'
        user_groups = df_temp.groupby('리뷰작성자')
        user_summary = user_groups.agg(
            X5_rating_mean=('작성자평균평점', 'first'),
            X6_review_num=('작성자리뷰작성수', 'first'),
        )
        print(f"[{platform_name}] 유저(X5, X6) 정보 집계 완료.")
        user_texts = user_groups['리뷰내용'].apply(list)
        print(f"[{platform_name}] 유저(X7) 유사도(Jaccard) 계산 시작...")
        user_summary['X7_content_sim'] = user_texts.apply(calculate_content_similarity)
        print(f"[{platform_name}] 유저(X7) 유사도(Jaccard) 계산 완료.")
    else:
        raise ValueError("Wrong platform name. Please enter 'naver' or 'kakao'")

    # --- X8: 바이럴 의심 여부 (median + k·IQR) ---
    # X8: 바이럴 의심 여부 피쳐 - 임계값 계산
    print(f"[{platform_name}] 바이럴(X8) 임계값 계산 시작...")
    store_daily_reviews = df_temp.groupby(
        ['가게이름', pd.Grouper(key=date_col_name, freq='D')]
    )['리뷰내용'].count()  # Series: index = (가게이름, 날짜), value = 일별 리뷰 수

    # 1) 가게별 median
    median = store_daily_reviews.groupby('가게이름').median().rename('median')

    # 2) 가게별 IQR (Q3 - Q1)
    q1 = store_daily_reviews.groupby('가게이름').quantile(0.25).rename('q1')
    q3 = store_daily_reviews.groupby('가게이름').quantile(0.75).rename('q3')
    iqr = (q3 - q1).rename('iqr')

    # 3) 하나의 DataFrame으로 결합
    store_stats = pd.concat([median, iqr], axis=1).fillna(0)

    # threshold = median + sigma * IQR
    store_stats['threshold'] = store_stats['median'] + sigma * store_stats['iqr']

    # 리뷰가 극히 적은 가게는 threshold=0으로 두어 X8 미사용
    min_days = store_daily_reviews.groupby('가게이름').size()
    store_stats.loc[min_days < 3, 'threshold'] = 0.0

    viral_thresholds = store_stats['threshold']
    print(f"[{platform_name}] 바이럴(X8) 임계값 계산 완료.")

    print(f"[{platform_name}] 바이럴(X8) 여부 계산 시작...")
    df_daily_counts = store_daily_reviews.to_frame(name='daily_count')
    df_thresholds = viral_thresholds.to_frame(name='threshold')

    df_viral_check = df_daily_counts.merge(
        df_thresholds,
        left_on='가게이름',
        right_index=True,
        how='left'
    )

    df_viral_check['X8_Is_Viral'] = (
            (df_viral_check['daily_count'] > df_viral_check['threshold']) &
            (df_viral_check['threshold'] > 0)
    )

    df_viral_flags = df_viral_check.reset_index().rename(
        columns={date_col_name: 'review_date_daily'}
    )
    df_viral_flags['review_date_daily'] = pd.to_datetime(
        df_viral_flags['review_date_daily']
    ).dt.normalize()
    df_viral_flags = df_viral_flags[['가게이름', 'review_date_daily', 'X8_Is_Viral']]
    print(f"[{platform_name}] 바이럴(X8) 여부 계산 완료.")

    # --- X1, X2, X4: 텍스트 기반 피처 ---
    print(f"[{platform_name}] 텍스트 피처(X1, X2, X4) 계산 시작...")
    vol_set = set(voluntary_keywords)
    ad_set  = set(ad_keywords)

    # 원 카운트
    df_temp['X1_Organic_Count'] = df_temp['리뷰내용'].apply(
        lambda x: count_keywords(x, vol_set, MORPH_ANALZYER)
    )
    df_temp['X2_Ad_Count'] = df_temp['리뷰내용'].apply(
        lambda x: count_keywords(x, ad_set, MORPH_ANALZYER)
    )

    # 텍스트 길이 (토큰 수 기준)로 정규화된 비율
    df_temp['token_len'] = df_temp['리뷰내용'].apply(
        lambda x: max(1, len(str(x).split()))
    )
    df_temp['X1_Organic_Ratio'] = df_temp['X1_Organic_Count'] / df_temp['token_len']
    df_temp['X2_Ad_Ratio']      = df_temp['X2_Ad_Count'] / df_temp['token_len']

    # 이진 플래그도 함께 보존
    df_temp['X1_Organic_Flag'] = (df_temp['X1_Organic_Count'] > 0).astype(int)
    df_temp['X2_Ad_Flag']      = (df_temp['X2_Ad_Count'] > 0).astype(int)

    df_temp['X4_Specificity'] = df_temp['리뷰내용'].apply(
        lambda x: len(str(x)) if pd.notna(x) else 0
    )
    print(f"[{platform_name}] 텍스트 피처(X1, X2, X4) 계산 완료.")

    # --- 피쳐 병합 ---
    print(f"[{platform_name}] 피처 병합 시작...")
    df_featured = df_temp.merge(user_summary, on='리뷰작성자', how='left')
    df_featured['review_date_daily'] = pd.to_datetime(
        df_featured[date_col_name]
    ).dt.normalize()
    df_featured = df_featured.merge(
        df_viral_flags,
        on=['가게이름', 'review_date_daily'],
        how='left'
    )
    df_featured['X8_Is_Viral'] = df_featured['X8_Is_Viral'].fillna(False).astype(int)
    df_featured = df_featured.drop(columns=['review_date_daily', 'token_len'])

    print(f"--- {platform_name} 피처 엔지니어링 완료. ---")
    return df_featured
# 피쳐 X3 및 X7 제외 피쳐 계산
def feature_missing_remover(df_naver, df_kakao) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    네이버/카카오 피쳐 결측치 및 저정보 행 처리:
      - X6_review_num == 0 이거나 NaN이면 행 삭제
      - Naver:
          X5_rating_mean: 결측 → 3.0 (비즈니스 룰)
          X7_content_sim: 결측 → 0.0
      - Kakao:
          X5_rating_mean: 결측 → median
          X7_content_sim: 결측 → 0.0
      - X1~X4는 0이 의미 있는 값이므로, NaN이 있으면 0으로 치환(방어적 처리)
    """
    n = df_naver.copy()
    k = df_kakao.copy()

    # --- 1) X6==0 또는 NaN 행 제거 ---
    if 'X6_review_num' in n.columns:
        before_n = len(n)
        n = n[n['X6_review_num'].notna() & (n['X6_review_num'] > 0)]
        print(f"[naver] X6_review_num==0 또는 NaN 행 제거: {before_n} → {len(n)}")

    if 'X6_review_num' in k.columns:
        before_k = len(k)
        k = k[k['X6_review_num'].notna() & (k['X6_review_num'] > 0)]
        print(f"[kakao] X6_review_num==0 또는 NaN 행 제거: {before_k} → {len(k)}")

    # --- 2) X5, X7 결측 처리 ---
    # Naver
    if 'X5_rating_mean' in n.columns:
        n['X5_rating_mean'] = n['X5_rating_mean'].fillna(3.0)
    if 'X7_content_sim' in n.columns:
        n['X7_content_sim'] = n['X7_content_sim'].fillna(0.0)

    # Kakao
    if 'X5_rating_mean' in k.columns:
        median_rating = k['X5_rating_mean'].median()
        k['X5_rating_mean'] = k['X5_rating_mean'].fillna(median_rating)
    if 'X7_content_sim' in k.columns:
        k['X7_content_sim'] = k['X7_content_sim'].fillna(0.0)

    # --- 3) X1~X4 방어적 NaN 처리 (있다면 0으로) ---
    for col in ['X1_Organic_Count', 'X1_Organic_Ratio',
                'X2_Ad_Count',      'X2_Ad_Ratio',
                'X1_Organic_Flag',  'X2_Ad_Flag',
                'X4_Specificity']:
        if col in n.columns:
            n[col] = n[col].fillna(0)
        if col in k.columns:
            k[col] = k[col].fillna(0)

    return n, k
# 피쳐 X5 ~ X7 결측치 처리

naver_reviews_filtered = pd.read_csv(CLEANED_DIR / 'Naver_Reviews_Filtered.csv', encoding='utf-8-sig')
kakao_reviews = pd.read_csv(CLEANED_DIR / 'Kakao_Reviews.csv', encoding='utf-8-sig')
naver_reviews_filtered['방문시간'] = naver_reviews_filtered['방문시간'].apply(
    parse_naver_visit_time_to_dt
)
kakao_reviews['리뷰작성일'] = kakao_reviews['리뷰작성일'].apply(
    parse_kakao_review_date_to_dt
)
naver_reviews_filtered['방문시간'] = pd.to_datetime(
    naver_reviews_filtered['방문시간'],
    format="%Y.%m.%d",
    errors="coerce",
)
kakao_reviews['리뷰작성일'] = pd.to_datetime(
    kakao_reviews['리뷰작성일'],
    format="%Y.%m.%d",
    errors="coerce",
)
with open(MODELS_DIR / "voluntary_keywords.pkl", "rb") as f:
    VOLUNTARY_PRESUMED_EXPAND = pickle.load(f)
with open(MODELS_DIR / "ad_keywords.pkl", "rb") as f:
    AD_SUBSPICION_EXPAND = pickle.load(f)
# 네이버: 필터링된 리뷰에 대해 피쳐 계산
naver_reviews_featured = feature_calculator(
    df_filtered=naver_reviews_filtered,
    sigma=1.5,
    platform_name='naver',
    ad_keywords=AD_SUBSPICION_EXPAND,
    voluntary_keywords=VOLUNTARY_PRESUMED_EXPAND,
)
# 카카오: 전처리된 전체 리뷰에 대해 피쳐 계산
kakao_reviews_featured = feature_calculator(
    df_filtered=kakao_reviews,
    sigma=1.5,
    platform_name='kakao',
    ad_keywords=AD_SUBSPICION_EXPAND,
    voluntary_keywords=VOLUNTARY_PRESUMED_EXPAND,
)
naver_reviews_featured, kakao_reviews_featured = feature_missing_remover(
    naver_reviews_featured,
    kakao_reviews_featured,
)
naver_reviews_featured.to_csv(CLEANED_DIR / 'Naver_Reviews_Featured.csv', encoding='utf-8-sig', index=False)
kakao_reviews_featured.to_csv(CLEANED_DIR / 'KaKao_Reviews_Featured.csv', encoding='utf-8-sig', index=False)

[naver] 유저(X5, X6) 정보 집계 완료.
[naver] 유저(X7) 유사도(Jaccard) 계산 시작...
[naver] 유저(X7) 유사도(Jaccard) 계산 완료.
[naver] 바이럴(X8) 임계값 계산 시작...
[naver] 바이럴(X8) 임계값 계산 완료.
[naver] 바이럴(X8) 여부 계산 시작...
[naver] 바이럴(X8) 여부 계산 완료.
[naver] 텍스트 피처(X1, X2, X4) 계산 시작...
[naver] 텍스트 피처(X1, X2, X4) 계산 완료.
[naver] 피처 병합 시작...
--- naver 피처 엔지니어링 완료. ---
[kakao] 유저(X5, X6) 정보 집계 완료.
[kakao] 유저(X7) 유사도(Jaccard) 계산 시작...
[kakao] 유저(X7) 유사도(Jaccard) 계산 완료.
[kakao] 바이럴(X8) 임계값 계산 시작...
[kakao] 바이럴(X8) 임계값 계산 완료.
[kakao] 바이럴(X8) 여부 계산 시작...
[kakao] 바이럴(X8) 여부 계산 완료.
[kakao] 텍스트 피처(X1, X2, X4) 계산 시작...
[kakao] 텍스트 피처(X1, X2, X4) 계산 완료.
[kakao] 피처 병합 시작...
--- kakao 피처 엔지니어링 완료. ---
[naver] X6_review_num==0 또는 NaN 행 제거: 29229 → 29229
[kakao] X6_review_num==0 또는 NaN 행 제거: 1841 → 1841


In [ ]:
# %% 강한 정답 준비
import pandas as pd
import re
from datetime import datetime
from collections import Counter
import pickle
import warnings
warnings.filterwarnings('ignore')

naver_reviews_labeled1 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-박세연.csv', encoding='utf-8-sig')
naver_reviews_labeled2 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-백진우.csv', encoding='utf-8-sig')
naver_reviews_labeled3 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-이상호.csv', encoding='utf-8-sig')
naver_reviews_labeled4 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-정지민.csv', encoding='utf-8-sig')
naver_reviews_stronglabeled = pd.concat([naver_reviews_labeled1, naver_reviews_labeled2, naver_reviews_labeled3, naver_reviews_labeled4])
kakao_reviews_stronglabeled = pd.read_csv(LABELED_DIR / 'Kakao_Reviews_StrongLabeled.csv', encoding='utf-8-sig')
# 1차 강한 정답 준비
naver_reviews_labeled1_2 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-박세연-2.csv', encoding='utf-8-sig')
naver_reviews_labeled2_2 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-백진우-2.csv', encoding='utf-8-sig')
naver_reviews_labeled3_2 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-이상호-2.csv', encoding='utf-8-sig')
naver_reviews_labeled4_2 = pd.read_csv(LABELED_DIR / 'Naver_Reviews_Sample-정지민-2.csv', encoding='utf-8-sig')
naver_reviews_stronglabeled_2 = pd.concat([naver_reviews_labeled1_2, naver_reviews_labeled2_2, naver_reviews_labeled3_2, naver_reviews_labeled4_2])
kakao_reviews_stronglabeled_2 = pd.read_csv(LABELED_DIR / 'Kakao_Reviews_StrongLabeled_2.csv', encoding='utf-8-sig')
# 2차 강한 정답 준비
naver_reviews_stronglabeled['방문시간'] = naver_reviews_stronglabeled['방문시간'].apply(
    parse_naver_visit_time_to_dt
)
kakao_reviews_stronglabeled['리뷰작성일'] = kakao_reviews_stronglabeled['리뷰작성일'].apply(
    parse_kakao_review_date_to_dt
)
kakao_reviews_stronglabeled_2['리뷰작성일'] = kakao_reviews_stronglabeled_2['리뷰작성일'].apply(
    parse_kakao_review_date_to_dt
)
naver_reviews_stronglabeled_2['방문시간'] = pd.to_datetime(
    naver_reviews_stronglabeled_2['방문시간'],
    errors="coerce"
)
mask_nat = naver_reviews_stronglabeled_2['방문시간'].isna()
naver_reviews_stronglabeled_2.loc[mask_nat, '방문시간'] = (
    naver_reviews_stronglabeled_2.loc[mask_nat, '방문시간']
        .astype(str)
        .apply(parse_naver_visit_time_to_dt)
)
naver_reviews_stronglabeled['방문시간'] = pd.to_datetime(
    naver_reviews_stronglabeled['방문시간'],
    format="%Y.%m.%d",
    errors="coerce",
)
kakao_reviews_stronglabeled['리뷰작성일'] = pd.to_datetime(
    kakao_reviews_stronglabeled['리뷰작성일'],
    format="%Y.%m.%d",
    errors="coerce",
)
kakao_reviews_stronglabeled_2['리뷰작성일'] = pd.to_datetime(
    kakao_reviews_stronglabeled_2['리뷰작성일'],
    format="%Y.%m.%d",
    errors="coerce",
)
# 강한 정답 데이터 방문시간, 리뷰작성일 컬럼 datetime으로 변환
naver_reviews_stronglabeled.to_csv(
    LABELED_DIR / 'Naver_Reviews_StrongLabeled.csv',
    index=False,
    encoding='utf-8-sig',
)
kakao_reviews_stronglabeled.to_csv(
    LABELED_DIR / 'Kakao_Reviews_StrongLabeled.csv',
    index=False,
    encoding='utf-8-sig',
)
naver_reviews_stronglabeled_2.to_csv(
    LABELED_DIR / 'Naver_Reviews_StrongLabeled_2.csv',
    index=False,
    encoding='utf-8-sig',
)
kakao_reviews_stronglabeled_2.to_csv(
    LABELED_DIR / 'Kakao_Reviews_StrongLabeled_2.csv',
    index=False,
    encoding='utf-8-sig',
)
# 병합된 강한 정답 데이터 저장

In [ ]:
# %% 광고성 리뷰 필터링 - 점수 가중치 및 점수 컷오프 최적화 (IBAS_DATAUP_ScoreOptimizer.py)
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_curve, f1_score, brier_score_loss
import joblib

def count_keywords(text, keyword_set, analyzer):
    if pd.isna(text) or not text:
        return 0
    try:
        cleaned_text = ALLOWED_PATTERN.sub(r' ', str(text))
        stripped_text = cleaned_text.strip()
        if not stripped_text:
            return 0
        nouns = analyzer.nouns(stripped_text)
        cnt = 0
        for noun in nouns:
            if noun in keyword_set:
                cnt += 1
        return cnt
    except Exception as e:
        print(f"Warning: Keyword count analysis failed: {e}")
        return 0

# 1) 피처 + 초기 weak label 구성
naver_reviews_featured = pd.read_csv(CLEANED_DIR / 'Naver_Reviews_Featured.csv', encoding='utf-8-sig')
kakao_reviews_featured = pd.read_csv(CLEANED_DIR / 'KaKao_Reviews_Featured.csv', encoding='utf-8-sig')

naver_reviews_featured['label'] = -1
kakao_reviews_featured['label'] = -1

naver_reviews_featured.loc[
    naver_reviews_featured['리뷰내용'].apply(
        lambda x: count_keywords(x, VOLUNTARY_CONFIRM_SEED, MORPH_ANALZYER)
    ) > 0,
    'label'
] = 0
kakao_reviews_featured.loc[
    kakao_reviews_featured['리뷰내용'].apply(
        lambda x: count_keywords(x, VOLUNTARY_CONFIRM_SEED, MORPH_ANALZYER)
    ) > 0,
    'label'
] = 0
kakao_reviews_featured.loc[kakao_reviews_featured['리뷰평점'] <= 3, 'label'] = 0

naver_reviews_featured.loc[
    naver_reviews_featured['리뷰내용'].apply(
        lambda x: count_keywords(x, AD_CONFIRM_SEED, MORPH_ANALZYER)
    ) > 0,
    'label'
] = 1
kakao_reviews_featured.loc[
    kakao_reviews_featured['리뷰내용'].apply(
        lambda x: count_keywords(x, AD_CONFIRM_SEED, MORPH_ANALZYER)
    ) > 0,
    'label'
] = 1

naver_reviews_featured['is_StrongLabeled'] = 0
kakao_reviews_featured['is_StrongLabeled'] = 0

print(naver_reviews_featured['label'].value_counts())
print(kakao_reviews_featured['label'].value_counts())

# 2) 1차 강한 정답 병합 (복합 키: 가게이름, 리뷰작성자, 날짜, 리뷰내용)
if os.path.exists(LABELED_DIR / 'Naver_Reviews_StrongLabeled.csv'):
    naver_reviews_stronglabeled = pd.read_csv(
        LABELED_DIR / 'Naver_Reviews_StrongLabeled.csv', encoding='utf-8-sig'
    )
    key_cols_naver = ['가게이름', '리뷰작성자', '방문시간', '리뷰내용']

    # strong 파일에서 key 기준으로 중복 제거 후, 신규 컬럼만 남기기
    naver_reviews_stronglabeled = naver_reviews_stronglabeled.drop_duplicates(subset=key_cols_naver)
    new_cols = [c for c in naver_reviews_stronglabeled.columns
                if c not in key_cols_naver and c not in naver_reviews_featured.columns]

    naver_reviews_scored = naver_reviews_featured.merge(
        naver_reviews_stronglabeled[key_cols_naver + new_cols],
        on=key_cols_naver,
        how='left'
    )

    naver_reviews_scored['label'] = np.where(
        naver_reviews_scored['validated_label'].notna(),
        naver_reviews_scored['validated_label'],
        naver_reviews_scored['label']
    )
    naver_reviews_scored['is_StrongLabeled'] = np.where(
        naver_reviews_scored['validated_label'].notna(),
        True,
        False
    )
else:
    naver_reviews_scored = naver_reviews_featured.copy()

if os.path.exists(LABELED_DIR / 'Kakao_Reviews_StrongLabeled.csv'):
    kakao_reviews_stronglabeled = pd.read_csv(
        LABELED_DIR / 'Kakao_Reviews_StrongLabeled.csv', encoding='utf-8-sig'
    )
    key_cols_kakao = ['가게이름', '리뷰작성자', '리뷰작성일', '리뷰내용']

    kakao_reviews_stronglabeled = kakao_reviews_stronglabeled.drop_duplicates(subset=key_cols_kakao)
    new_cols = [c for c in kakao_reviews_stronglabeled.columns
                if c not in key_cols_kakao and c not in kakao_reviews_featured.columns]

    kakao_reviews_scored = kakao_reviews_featured.merge(
        kakao_reviews_stronglabeled[key_cols_kakao + new_cols],
        on=key_cols_kakao,
        how='left'
    )

    kakao_reviews_scored['label'] = np.where(
        kakao_reviews_scored['validated_label'].notna(),
        kakao_reviews_scored['validated_label'],
        kakao_reviews_scored['label']
    )
    kakao_reviews_scored['is_StrongLabeled'] = np.where(
        kakao_reviews_scored['validated_label'].notna(),
        True,
        False
    )
else:
    kakao_reviews_scored = kakao_reviews_featured.copy()

print(naver_reviews_scored['is_StrongLabeled'].value_counts())
print(kakao_reviews_scored['is_StrongLabeled'].value_counts())

# 3) 2차 강한 정답 병합 (동일 복합 키 사용)
if os.path.exists(LABELED_DIR / 'Naver_Reviews_StrongLabeled_2.csv'):
    naver_strong2 = pd.read_csv(
        LABELED_DIR / 'Naver_Reviews_StrongLabeled_2.csv',
        encoding='utf-8-sig'
    )
    key_cols_naver = ['가게이름', '리뷰작성자', '방문시간', '리뷰내용']
    cols_keep = key_cols_naver + ['validated_label']

    naver_strong2 = (
        naver_strong2[cols_keep]
        .drop_duplicates(subset=key_cols_naver)
        .rename(columns={'validated_label': 'validated_label_round2'})
    )

    naver_reviews_scored = naver_reviews_scored.merge(
        naver_strong2,
        on=key_cols_naver,
        how='left'
    )

    naver_reviews_scored['label'] = np.where(
        naver_reviews_scored['validated_label_round2'].notna(),
        naver_reviews_scored['validated_label_round2'],
        naver_reviews_scored['label']
    )
    naver_reviews_scored['is_StrongLabeled'] = np.where(
        naver_reviews_scored['validated_label_round2'].notna(),
        True,
        naver_reviews_scored['is_StrongLabeled']
    )

    naver_reviews_scored = naver_reviews_scored.drop(columns=['validated_label_round2'])

if os.path.exists(LABELED_DIR / 'Kakao_Reviews_StrongLabeled_2.csv'):
    kakao_strong2 = pd.read_csv(
        LABELED_DIR / 'Kakao_Reviews_StrongLabeled_2.csv',
        encoding='utf-8-sig'
    )
    key_cols_kakao = ['가게이름', '리뷰작성자', '리뷰작성일', '리뷰내용']
    cols_keep = key_cols_kakao + ['validated_label']

    kakao_strong2 = (
        kakao_strong2[cols_keep]
        .drop_duplicates(subset=key_cols_kakao)
        .rename(columns={'validated_label': 'validated_label_round2'})
    )

    kakao_reviews_scored = kakao_reviews_scored.merge(
        kakao_strong2,
        on=key_cols_kakao,
        how='left'
    )

    kakao_reviews_scored['label'] = np.where(
        kakao_reviews_scored['validated_label_round2'].notna(),
        kakao_reviews_scored['validated_label_round2'],
        kakao_reviews_scored['label']
    )
    kakao_reviews_scored['is_StrongLabeled'] = np.where(
        kakao_reviews_scored['validated_label_round2'].notna(),
        True,
        kakao_reviews_scored['is_StrongLabeled']
    )

    kakao_reviews_scored = kakao_reviews_scored.drop(columns=['validated_label_round2'])

print(naver_reviews_scored['is_StrongLabeled'].value_counts())
print(kakao_reviews_scored['is_StrongLabeled'].value_counts())
# 강한 정답 데이터 로드 후 기존 데이터에 병합
# validated_label이 NaN이 아닌 경우, 'label'을 덮어쓰고 'is_StrongLabeled'을 True로 변경

feature_cols = [
    'X1_Organic_Ratio',
    'X2_Ad_Ratio',
    'X4_Specificity',
    'X5_rating_mean', 'X6_review_num', 'X7_content_sim',
    'X8_Is_Viral',
    'platform_is_naver',
]

naver_reviews_scored['platform_is_naver'] = 1
kakao_reviews_scored['platform_is_naver'] = 0

naver_train = naver_reviews_scored[
    feature_cols + ['label', 'is_StrongLabeled']
].copy()
naver_train['platform_is_naver'] = 1

kakao_train = kakao_reviews_scored[
    feature_cols + ['label', 'is_StrongLabeled']
].copy()
kakao_train['platform_is_naver'] = 0

train_df = pd.concat([naver_train, kakao_train], ignore_index=True)
train_df = train_df[train_df['label'] != -1].copy()
train_df['label'] = train_df['label'].astype(int)

positive_rate = train_df['label'].mean()
print(f"[XGB] 학습 데이터 기준 광고 레이블 비율: {positive_rate:.4f}")

X_all = train_df[feature_cols].values
y_all = train_df['label'].values
is_strong = train_df['is_StrongLabeled'].values.astype(bool)

# ===== 강한 정답 weight 스윕 설정 =====
strong_weights = [0.2, 0.25, 0.33, 0.5, 1.0, 2.0, 3.0, 4,0, 5.0]

best_brier = np.inf
best_result = None  # (weight, xgb, calibrated, best_threshold, precision, recall, f1)

for sw in strong_weights:
    print(f"\n=== strong_weight = {sw:.1f} ===")

    # sample_weight 구성: strong → sw, weak → 1.0
    w_all = np.where(is_strong, sw, 1.0).astype(np.float32)

    X_train, X_valid, y_train, y_valid, w_train, w_valid = train_test_split(
        X_all, y_all, w_all,
        test_size=0.2,
        random_state=2025,
        stratify=y_all
    )

    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train)
    if np.issubdtype(y_train.dtype, np.floating):
        y_train = y_train.astype(np.int64)
    w_train = np.asarray(w_train, dtype=np.float32)

    xgb = XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        eval_metric='logloss',
        random_state=2025,
        n_jobs=-1,
    )

    xgb.fit(
        X_train, y_train,
        sample_weight=w_train,
        eval_set=[(X_valid, y_valid)],
        verbose=False,
    )

    calibrated = CalibratedClassifierCV(
        estimator=xgb,
        method='sigmoid',
        cv=5,
    )
    calibrated.fit(X_train, y_train, sample_weight=w_train)

    valid_probs = calibrated.predict_proba(X_valid)[:, 1]

    brier = brier_score_loss(y_valid, valid_probs)
    print(f"[XGB][sw={sw:.1f}] 검증 셋 Brier score: {brier:.4f}")

    precision, recall, thresholds = precision_recall_curve(y_valid, valid_probs)
    # precision, recall 길이는 thresholds보다 1 더 깁니다.
    # thresholds[k] 에 대응하는 precision/recall은 precision[k+1], recall[k+1].

    # Precision ≥ prec_constraint 조건을 만족하는 인덱스들 선택
    prec_constraint = 0.45

    # thresholds 인덱스(k)에 대응하는 precision/recall 인덱스는 k+1
    valid_idxs = []
    for k in range(len(thresholds)):
        p = precision[k + 1]
        r = recall[k + 1]
        if p >= prec_constraint:
            valid_idxs.append((k, p, r))

    if not valid_idxs:
        # Precision ≥ prec_constraint를 만족하는 threshold가 없다면,
        # fallback: 전체에서 Recall이 최대인 threshold를 사용 (혹은 F1 최대 등)
        print(f"[XGBoost][sw={sw:.1f}] Precision ≥ {prec_constraint} 만족 threshold 없음, "
              "fallback으로 Recall 최대 threshold 사용")
        best_idx = np.argmax(recall[:-1])  # 마지막은 dummy라 제외
    else:
        # Precision ≥ prec_constraint인 후보들 중 Recall이 최대인 threshold 선택
        best_k, best_p, best_r = max(valid_idxs, key=lambda x: x[2])
        best_idx = best_k
        print(f"[XGBoost][sw={sw:.1f}] Precision ≥ {prec_constraint} 조건에서 "
              f"Recall 최대 threshold 선택 (P={best_p:.3f}, R={best_r:.3f})")

    best_threshold = thresholds[best_idx]

    print(f"[XGBoost][sw={sw:.1f}] 선택된 threshold: {best_threshold:.3f}")
    print(f"  - Precision: {precision[best_idx + 1]:.3f}")
    print(f"  - Recall   : {recall[best_idx + 1]:.3f}")
    # F1은 참고용으로 계산
    best_f1 = 2 * (precision[best_idx + 1] * recall[best_idx + 1]) / \
              (precision[best_idx + 1] + recall[best_idx + 1] + 1e-8)
    print(f"  - F1       : {best_f1:.3f}")

    # Brier 기준 best weight 갱신 로직은 그대로 유지
    if brier < best_brier:
        best_brier = brier
        best_result = {
            'strong_weight': sw,
            'xgb': xgb,
            'calibrated': calibrated,
            'best_threshold': float(best_threshold),
            'precision': float(precision[best_idx + 1]),
            'recall': float(recall[best_idx + 1]),
            'f1': float(best_f1),
        }

# ===== 최종 best weight 및 모델 사용 =====
print("\n=== 최적 strong_weight 선택 결과 ===")
print(f"  strong_weight: {best_result['strong_weight']:.1f}")
print(f"  검증 Brier    : {best_brier:.4f}")
print(f"  최적 threshold: {best_result['best_threshold']:.3f}")
print(f"  Precision     : {best_result['precision']:.3f}")
print(f"  Recall        : {best_result['recall']:.3f}")
print(f"  F1            : {best_result['f1']:.3f}")

best_xgb = best_result['xgb']
best_calibrated = best_result['calibrated']
best_threshold = best_result['best_threshold']

def add_scores(df, platform_flag):
    df = df.copy()
    df['platform_is_naver'] = platform_flag
    X_all_df = df[feature_cols].values
    probs = best_calibrated.predict_proba(X_all_df)[:, 1]
    preds = (probs >= best_threshold).astype(int)
    df['ad_prob'] = probs
    df['ad_pred'] = preds
    df = df.drop(columns=['platform_is_naver'])
    return df

naver_reviews_scored = add_scores(naver_reviews_scored, platform_flag=1)
kakao_reviews_scored  = add_scores(kakao_reviews_scored,  platform_flag=0)

naver_reviews_scored.to_csv(
    SCORED_DIR / 'Naver_Reviews_AdScored.csv',
    index=False,
    encoding='utf-8-sig',
)
kakao_reviews_scored.to_csv(
    SCORED_DIR / 'Kakao_Reviews_AdScored.csv',
    index=False,
    encoding='utf-8-sig',
)

model_bundle = {
    'feature_cols': feature_cols,
    'xgb_base': best_xgb,
    'calibrated_model': best_calibrated,
    'best_threshold': best_threshold,
    'strong_weight': best_result['strong_weight'],
}
joblib.dump(model_bundle, MODELS_DIR / 'xgb_ad_detector.joblib')
print(f"✓ 광고성 리뷰 XGBoost 모델 및 threshold 저장 완료: {MODELS_DIR / 'xgb_ad_detector.joblib'}")

label
-1    28835
 0      295
 1       99
Name: count, dtype: int64
label
-1    1239
 0     594
 1       8
Name: count, dtype: int64
is_StrongLabeled
False    28398
True       831
Name: count, dtype: int64
is_StrongLabeled
False    1755
True       86
Name: count, dtype: int64
is_StrongLabeled
False    27039
True      2190
Name: count, dtype: int64
is_StrongLabeled
False    1670
True      171
Name: count, dtype: int64
[XGB] 학습 데이터 기준 광고 레이블 비율: 0.2077

=== strong_weight = 0.2 ===
[XGB][sw=0.2] 검증 셋 Brier score: 0.1381
[XGBoost][sw=0.2] Precision ≥ 0.45 조건에서 Recall 최대 threshold 선택 (P=0.452, R=0.593)
[XGBoost][sw=0.2] 선택된 threshold: 0.248
  - Precision: 0.452
  - Recall   : 0.593
  - F1       : 0.513

=== strong_weight = 0.2 ===
[XGB][sw=0.2] 검증 셋 Brier score: 0.1363
[XGBoost][sw=0.2] Precision ≥ 0.45 조건에서 Recall 최대 threshold 선택 (P=0.450, R=0.637)
[XGBoost][sw=0.2] 선택된 threshold: 0.233
  - Precision: 0.450
  - Recall   : 0.637
  - F1       : 0.528

=== strong_weight = 0.3 ===
[XGB][sw=0.3

In [ ]:
# %% 광고성 리뷰 필터링 - 점수 계산 (IBAS_DATAUP_ScoreCalculator.py)
# 반복성 점수 계산 호출에 효율적으로 대응
import pandas as pd
import joblib

model_bundle = joblib.load(MODELS_DIR / 'xgb_ad_detector.joblib')
feature_cols    = model_bundle['feature_cols']
calibrated_clf  = model_bundle['calibrated_model']
best_threshold  = model_bundle['best_threshold']
# 모델 번들 로딩
def score_reviews_with_xgb(df_featured, platform_name: str) -> pd.DataFrame:
    """
    피쳐 계산 완료된 리뷰 DataFrame과 플랫폼명을 받아
    xgb_ad_detector 모델로 광고 확률(ad_prob)과 이진 예측(ad_pred)을 추가해 반환.
      - platform_name: 'naver' 또는 'kakao'
      - df_featured: X1~X8 등 피쳐가 모두 포함된 DataFrame
    """
    df = df_featured.copy()

    # 플랫폼 플래그 추가 (학습 때와 동일하게 맞춰야 함)
    if 'platform_is_naver' not in df.columns:
        if platform_name.lower() == 'naver':
            df['platform_is_naver'] = 1
        elif platform_name.lower() == 'kakao':
            df['platform_is_naver'] = 0
        else:
            raise ValueError("platform_name must be 'naver' or 'kakao'")

    # 피쳐 컬럼 존재 여부 체크
    missing = [c for c in feature_cols if c not in df.columns]
    if missing:
        raise ValueError(f"다음 피쳐 컬럼이 DataFrame에 없습니다: {missing}")

    x = df[feature_cols].values

    # 보정된 확률 예측 (광고성 y=1 클래스 확률)
    probs = calibrated_clf.predict_proba(x)[:, 1]

    df['ad_prob'] = probs
    df['ad_pred'] = (probs >= best_threshold).astype(int)
    return df
# 이미 최정화된 모델로 광고성 확률 계산
naver_featured = pd.read_csv(CLEANED_DIR / 'Naver_Reviews_Featured.csv', encoding='utf-8-sig')
kakao_featured = pd.read_csv(CLEANED_DIR / 'KaKao_Reviews_Featured.csv', encoding='utf-8-sig')
naver_scored = score_reviews_with_xgb(naver_featured, platform_name='naver')
kakao_scored = score_reviews_with_xgb(kakao_featured, platform_name='kakao')
naver_scored.to_csv(SCORED_DIR / 'Naver_Reviews_AdScored.csv', index=False, encoding='utf-8-sig')
kakao_scored.to_csv(SCORED_DIR / 'Kakao_Reviews_AdScored.csv', index=False, encoding='utf-8-sig')
print(naver_scored['ad_prob'].describe())
print(naver_scored['ad_pred'].value_counts())
print(kakao_scored['ad_prob'].describe())
print(kakao_scored['ad_pred'].value_counts())
print("✓ 광고성 리뷰 확률(ad_prob) 및 예측(ad_pred) 계산/저장 완료.")

count    29229.000000
mean         0.113938
std          0.079344
min          0.067992
25%          0.072899
50%          0.081183
75%          0.115481
max          0.683391
Name: ad_prob, dtype: float64
ad_pred
0    17405
1    11824
Name: count, dtype: int64
count    1841.000000
mean        0.077391
std         0.024822
min         0.067643
25%         0.068058
50%         0.069755
75%         0.077877
max         0.498169
Name: ad_prob, dtype: float64
ad_pred
0    1609
1     232
Name: count, dtype: int64
✓ 광고성 리뷰 확률(ad_prob) 및 예측(ad_pred) 계산/저장 완료.
